In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 한글처리
import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname="C:\\Windows\\Fonts\\malgun.ttf").get_name()
plt.rc("font", family=font_name)

# 음수 - 표시 처리
import matplotlib as mlp
mlp.rcParams["axes.unicode_minus"] = False

# 주제 : 서울시 구별 범죄 현황 분석

## 부제

+ CCTV와 범죄, 검거율이 관계가 있는가 ?
+ 강남3구는 과연 범죄 발생률이 낮고 검거율이 높은가 ?
+ 우리동네는 어떤가 ?

## 1. Data Prepare

+ https://data.go.kr
+ 구글에서 키워드 "서울시 관서별 5대범죄 발생 검거 현황" 로 검색

In [9]:
crime_anal_police = pd.read_csv("data/seoul/crime_in_seoul.csv", encoding="euc-kr", thousands=",")
crime_anal_police.head()

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거
0,중부서,2,2,3,2,105,65,1395,477,1355,1170
1,종로서,3,3,6,5,115,98,1070,413,1278,1070
2,남대문서,1,0,6,4,65,46,1153,382,869,794
3,서대문서,2,2,5,4,154,124,1812,738,2056,1711
4,혜화서,3,2,5,4,96,63,1114,424,1015,861


In [10]:
# 위의 숫자에서 천단위 , 로 인해 타입이 int가 아닌 object 로 되있음, 그래서 read_csv의 속성중 하나인 thousands 에 , 를 지정해주면
# 읽어올 때 이를 int로 변경해 준다.

crime_anal_police.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   관서명     31 non-null     object
 1   살인 발생   31 non-null     int64 
 2   살인 검거   31 non-null     int64 
 3   강도 발생   31 non-null     int64 
 4   강도 검거   31 non-null     int64 
 5   강간 발생   31 non-null     int64 
 6   강간 검거   31 non-null     int64 
 7   절도 발생   31 non-null     int64 
 8   절도 검거   31 non-null     int64 
 9   폭력 발생   31 non-null     int64 
 10  폭력 검거   31 non-null     int64 
dtypes: int64(10), object(1)
memory usage: 2.8+ KB


## 2. EDA & Preprocessing

### 2.1 관서명을 구 이름으로 변경

+ https://ko.wikipedia.org/wiki/서울특별시지방경찰청

+ kakao API를 이용하여 주소 획득
    - https://developers.kakao.com

In [13]:
# 서버에 요청을 하기 위한 모듈

import requests

header = {"Authorization":"KakaoAK 872d679b7aa257eb46c4bc7bac47e7fe"}
url = "https://dapi.kakao.com/v2/local/search/keyword.json?query='중부경찰서'"

requests.get(url, headers=header).json()

{'documents': [{'address_name': '서울 중구 저동2가 62-1',
   'category_group_code': 'PO3',
   'category_group_name': '공공기관',
   'category_name': '사회,공공기관 > 행정기관 > 경찰서',
   'distance': '',
   'id': '11151738',
   'phone': '182',
   'place_name': '서울중부경찰서',
   'place_url': 'http://place.map.kakao.com/11151738',
   'road_address_name': '서울 중구 수표로 27',
   'x': '126.98958691395208',
   'y': '37.56361812722828'},
  {'address_name': '경기 수원시 장안구 정자동 873-4',
   'category_group_code': 'PO3',
   'category_group_name': '공공기관',
   'category_name': '사회,공공기관 > 행정기관 > 경찰서',
   'distance': '',
   'id': '16176863',
   'phone': '182',
   'place_name': '수원중부경찰서',
   'place_url': 'http://place.map.kakao.com/16176863',
   'road_address_name': '경기 수원시 장안구 정자천로 199',
   'x': '126.99627805098389',
   'y': '37.29789068092514'},
  {'address_name': '부산 중구 대창동1가 40',
   'category_group_code': 'PO3',
   'category_group_name': '공공기관',
   'category_name': '사회,공공기관 > 행정기관 > 경찰서',
   'distance': '',
   'id': '10465693',
   'p